# <span style="color:orange"> *Multiple bounded optimization* </span> 

In [1]:
%matplotlib inline
import nibabel as nib
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import subprocess
import sys
import pandas as pd
import os
from utils.tools import chi_to_fm_ppm

 # <span style="color:lime"> *Repository & Tool loading* </span> 

In [2]:
%cd susceptibility-to-fieldmap-fft 
!git status
%pip install .

c:\Users\User\msc_project\sus-optimization\susceptibility-to-fieldmap-fft


c:\Users\User\miniconda3\envs\saiteki2\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


HEAD detached at d9f785b
nothing to commit, working tree clean
Processing c:\users\user\msc_project\sus-optimization\susceptibility-to-fieldmap-fft
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for susceptibility-to-fieldmap-fft: filename=susceptibility_to_fieldmap_fft-0.0.0-py3-none-any.whl size=23543 sha256=3583f74da4e25e182159d91e1b86240873ac7c7b28cccf24f1fac65a03c7a5b5
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\76\2b\f9\862133fb7ad25c2aad966eb13c8d8caeb255ceff7b9ed004ca
Successfully built susceptibility-to-fieldmap-fft
  Attempting uninstall: susceptibility-to-fieldmap-fft
    Found existing installation: susceptibility-to-fieldmap-fft 0.0.0
    Uninstalling susceptibility-to-fieldmap-fft-0.0.0:
      Successfully uninstalled susceptibility-to-fieldmap-fft-0.0.0
Note: you may need to restart the kernel to use updated packages.


Sometimes it is necessary to go to the home folder to load the utils folder

In [3]:
home_path = "C:/Users/User/msc_project/sus-optimization"
#Once we confirmed the head of the chi to fbfest we can go to home
%cd {home_path}
!ls

C:\Users\User\msc_project\sus-optimization
README.md
__pycache__
chi_opt_pro.ipynb
data
evas_code.ipynb
fixed_chi_test.ipynb
internal_air_only.nii.gz
output
requirements.txt
simulation
soothsayer.ipynb
spinal_cord_voxels_only.nii.gz
sus_optimization.ipynb
susceptibility-to-fieldmap-fft
tissue-to-MRproperty
utils


In [4]:
path_to_chi_to_fm_fft = "C:/Users/User/msc_project/susceptibility-to-fieldmap-fft"
sys.path.append(path_to_chi_to_fm_fft)
#sys.path.append('/Users/evaalonsoortiz/Documents/python/susceptibility-to-fieldmap-fft/')
from functions import compute_fieldmap

## <span style="color:lime"> *Util functions used during the optimization process* </span> 

In [5]:
# We need to reate the custom command that creates the fieldmap
from utils.tools import chi_to_fm_ppm
# Input is a file and outpit is the file
# Ex: chi_to_fm_ppm(in_file_path,out_file_path)

In [6]:
# Function to automate the subprocess of extractiing measure with SCT through the terminal
from utils.extract_metric import extract_metrics
# Example of calling the function:
#extract_metrics(path_to_dmod_fm, path_to_dmod_mask, path_metric_mask, out_B0_metrics_path, method='wa', vert="3:15") # Change numbers x:y accordingly

 # <span style="color:gold"> *Optimization begins!* </span> 
 Remember to change the folder name of iteration demod and metrics so that you don't mix runs! </br>
 Code runs anywhere between an hour to hour and a half depending on the range of values and size of image

In [7]:
# First lets define the paths for simulation files
path_to_iter_dmod_fms =  "E:/msc_data/ismrm_2025/db0_032/fm/chi-opt/iteration_fms/run5"
path_to_iter_metrics =  "E:/msc_data/ismrm_2025/db0_032/fm/chi-opt/iteration_metrics/run5"

path_to_sim_metric_mask = "E:/msc_data/ismrm_2025/db0_032/fm/sim/D2_D3_masks/cord_mask_labeled.nii.gz"
path_to_dmod_mask = "E:/msc_data/ismrm_2025/db0_032/fm/sim/D2_D3_masks/cord_mask.nii.gz"
path_to_chimap = "E:/msc_data/ismrm_2025/db0_032/fm/sim/B1_chi_maps/db0_032_sus_opt_map.nii.gz" # -4.36 (past value)
path_to_segs = "E:/msc_data/ismrm_2025/db0_032/fm/chi-opt/grouped_wb_segs.nii.gz"


counter = 0

In [8]:
# Now load the measured values (done 1 time outside of loop, this values are the reference)
b0meas_metrics = pd.read_csv("E:/msc_data/ismrm_2025/db0_032/fm/C_dmod_meas/exp/exp_metrics.csv")
dmod_b0meas_vert_values = b0meas_metrics["WA()"]
dmod_b0meas_vert_values = dmod_b0meas_vert_values[::-1]
dmod_b0meas_vert_values

12   -133.966738
11   -121.118786
10    -80.314337
9      -8.943227
8      55.167182
7      39.844835
6     -12.995903
5      -8.459881
4      22.489457
3      56.544607
2      63.354887
1      41.714478
0      19.229226
Name: WA(), dtype: float64

In [9]:
def f_max(x):

    global counter # This works? 
    counter += 1  
    print('################')
    print(f"Iteration #{counter}")
    print(f"Current values of variables: x[0] = {x[0]} & x[1] = {x[1]}")
    #print(f"Current values of variables: x[0] = {x[0]}, x[1] = {x[1]} & x[2] = {x[2]}")

    # Load the simulated susceptibility map in ppm
    sim_chi_img = nib.load(path_to_chimap)
    sim_chi_data = sim_chi_img.get_fdata()

    # Load segmentation labels that create the chimaps
    ROI_img = nib.load(path_to_segs)
    ROI_data = ROI_img.get_fdata()

    # Now we load the demodulation mask, which in this case is also the Spinal cord mask, can change
    dmod_mask_img = nib.load(path_to_dmod_mask)
    dmod_mask_data = dmod_mask_img.get_fdata()

    # Find indices with the labels we want to iterate!
    ind_trachea = np.where((ROI_data == 8))
    ind_lungs = np.where((ROI_data == 7))
    ind_body = np.where((ROI_data == 2))
    ind_bone = np.where((ROI_data == 3))
    
    # Assign variables 
    sim_chi_data[ind_trachea] = x[0]
    sim_chi_data[ind_lungs] = x[1]
    #sim_chi_data[ind_body] = x[2]
    #sim_chi_data[ind_bone] = x[3]

    # Now we gotta compute the B0 distribution in [ppm] using compute_bz from repository
    # Lets load the chi map with load_sus_dist from compute fieldmap to get the image res for FBFest
    chi_dist, image_res, affine_matrix = compute_fieldmap.load_sus_dist(path_to_chimap)

    sim_b0_ppm =  compute_fieldmap.compute_bz(sim_chi_data, image_resolution = image_res, buffer = 50,mode = "b0SimISMRM")
    # bz will be in [ppm] we go to Hz now
    sim_b0_Hz = sim_b0_ppm * 3 * 42.58 # deltaB0 [Hz] = dB0 [ppm] * 1e-6  * B0 [T] * 42.58* e6 [Hz/T] 

    # Chi[0] trachea and Chi[1] lung
    chi1 = x[0]
    chi2 = x[1]
    #chi3 = x[2]

    chi1_name = str(str(float(f"{chi1:.3f}"))).replace(".","_") # to take away the minus sign can use .strip("-")) at the end
    chi2_name = str(str(float(f"{chi2:.3f}"))).replace(".","_")
    #chi3_name = str(str(float(f"{chi3:.3f}"))).replace(".","_")

    dmod_fm_name = f"dmod_sim_fm_tr_{chi1_name}_l_{chi2_name}_Hz.nii.gz" # First lets try 2, then 3 different for body with IMC prior
    #dmod_fm_name = f"dmod_sim_fm_tr_{chi1_name}_l_{chi2_name}_bd_{chi3_name}_Hz.nii.gz"
    path_to_dmod_fm = os.path.join(path_to_iter_dmod_fms, dmod_fm_name)

    # Now we need to demodulated the fieldmap 

    dmod_value = np.mean(sim_b0_Hz[dmod_mask_data == 1])
    print("Demod value for Simulation: ", dmod_value)
    dmod_sim_b0_Hz = sim_b0_Hz - dmod_value

    dmod_sim_b0_Hz_img = nib.Nifti1Image(dmod_sim_b0_Hz, affine = sim_chi_img.affine)
    nib.save(dmod_sim_b0_Hz_img, path_to_dmod_fm)
    print("Saving ... ", dmod_fm_name)

    # Now we extract metrics 
    metrics_name = f"metrics_sim_fm_tr_{chi1_name}_l_{chi2_name}_Hz.csv"
    #metrics_name = f"metrics_sim_fm_tr_{chi1_name}_l_{chi2_name}_bd_{chi3_name}_Hz.csv"
    path_to_out_metrics = os.path.join(path_to_iter_metrics,metrics_name)

    extract_metrics(path_to_dmod_fm, path_to_dmod_mask, path_to_sim_metric_mask, path_to_out_metrics, method='wa', vert="3:15")

    metric_values = pd.read_csv(path_to_out_metrics)
    dmod_sim_vert_values = metric_values['WA()']
    dmod_sim_vert_values = dmod_sim_vert_values[::-1]

    difference = np.linalg.norm(dmod_b0meas_vert_values - dmod_sim_vert_values )
    print("Minimizing difference: ", difference)

    return difference
    


In [10]:
# Set initial values, boundaries and run optimization
initial_values = [-1, -4.36] # Body should begin
# First bound is trachea
# Second bound is Lung
bounds = [(-2,0), (-9, 1)]

if counter != 0 :
        # This means that you forgot to change the folder run number, to avoid mixing tests, please run that cell 
        # Changing the number after run!
    print("Please change run # to avoid mixing result folders :)")
else:
    if not os.path.exists(path_to_iter_dmod_fms) and os.path.exists(path_to_iter_metrics):
        os.makedirs(path_to_iter_dmod_fms)
        os.makedirs(path_to_iter_metrics)
        print("Folders created: ", path_to_iter_dmod_fms)
    result = optimize.minimize(f_max, initial_values, bounds = bounds, method = 'Nelder-Mead')
    # Output the result
    print("Optimization result:", result)

################
Iteration #1
Current values of variables: x[0] = -1.0 & x[1] = -4.36
Demod value for Simulation:  -270.5205571067217
Saving ...  dmod_sim_fm_tr_-1_0_l_-4_36_Hz.nii.gz
Metrics extracted successfully: 
--
Spinal Cord Toolbox (6.2)

sct_extract_metric -i E:/msc_data/ismrm_2025/db0_032/fm/chi-opt/iteration_fms/run5\dmod_sim_fm_tr_-1_0_l_-4_36_Hz.nii.gz -f E:/msc_data/ismrm_2025/db0_032/fm/sim/D2_D3_masks/cord_mask.nii.gz -method wa -vert 3:15 -vertfile E:/msc_data/ismrm_2025/db0_032/fm/sim/D2_D3_masks/cord_mask_labeled.nii.gz -perlevel 1 -o E:/msc_data/ismrm_2025/db0_032/fm/chi-opt/iteration_metrics/run5\metrics_sim_fm_tr_-1_0_l_-4_36_Hz.csv
--


Load metric image...
Estimation for label: cord_mask

Done! To view results, type:
start E:/msc_data/ismrm_2025/db0_032/fm/chi-opt/iteration_metrics/run5\metrics_sim_fm_tr_-1_0_l_-4_36_Hz.csv


Minimizing difference:  191.17028577014844
################
Iteration #2
Current values of variables: x[0] = -1.05 & x[1] = -4.36
Demod va

# <span style="color:chocolate"> *Plot and compare!* </span> 

In [ ]:
run5_ = d.read_csv("")
run5_values = run5_["WA()"]
run5_values = run5_values[::-1]